In [3]:
# 1. 패키지 임포트
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from oledpy.dipole_emission import ThinFilmArchitecture, Layer
from oledpy.utils import load_nk_data
import warnings
warnings.filterwarnings('ignore')

# 2. 데이터 경로 설정
BASE_DIR = os.path.dirname(os.path.abspath(__file__))  # 코드 파일 위치 기준
PL_PATH = os.path.join(BASE_DIR, "Reference Data", "tesu_lee_PL_value.xlsx")
NK_PATH = os.path.join(BASE_DIR, "Reference Data", "tesu_lee_nk_value.xlsx")

# 3. 데이터 로드 함수
def load_data():
    """PL 및 NK 데이터 로드 및 검증"""
    # PL 데이터
    df_pl = pd.read_excel(PL_PATH, sheet_name="Sheet1")
    print("PL 데이터 검증 완료:", df_pl[["wavelength_blue", "abs_blue"]].head())
    
    # NK 데이터
    nk_df = load_nk_data(NK_PATH)  # OLEDpy 전용 로더
    required_columns = ["QD_n", "QD_k"]
    for col in required_columns:
        if col not in nk_df.columns:
            raise KeyError(f"{col} 컬럼 누락")
    print("NK 데이터 검증 완료:", nk_df.columns.tolist())
    
    return df_pl, nk_df

# 4. 레이어 구성 함수
def build_layers(etl_thickness=40):
    """ETL 두께에 따른 레이어 구조 생성"""
    return [
        Layer(name='Ag', thickness=100, material='Ag', coherent=True),
        Layer(name='MoO3', thickness=10, material='MoO3', coherent=True),
        Layer(name='TCTA', thickness=35, material='TCTA', coherent=True),
        Layer(name='QD', thickness=25, material='QD', coherent=True, is_active=True),
        Layer(name='ZnO', thickness=etl_thickness, material='ZnO', coherent=True),
        Layer(name='Ag', thickness=15, material='Ag', coherent=True)
    ]

# 5. 광학 시뮬레이션 엔진
class TQLEDSimulator:
    def __init__(self, vac_wavelengths=np.arange(300, 501, 5)):
        self.df_pl, self.nk_df = load_data()
        self.vac_wavelengths = vac_wavelengths
        
        # PL 스펙트럼 보간 (Blue 기준)
        self.pl_spectrum = np.interp(
            vac_wavelengths,
            self.df_pl["wavelength_blue"],
            self.df_pl["abs_blue"]
        )
        
    def run_simulation(self, d_zn):
        """단일 ETL 두께 시뮬레이션 실행"""
        layers = build_layers(d_zn)
        
        arch = ThinFilmArchitecture(
            layers=layers,
            dipole_positions=[0.9],
            tau=1e-6,
            PLQY=0.95,
            vac_wavelengths=self.vac_wavelengths,
            pl_spectrum=self.pl_spectrum
        )
        arch.load_material_data(self.nk_df)
        arch.calculate_optics()
        return arch.calc_CE(current_density=10)  # J=10 mA/cm²

# 6. 2D 전력 소산 스펙트럼 시각화
def plot_power_dissipation(arch):
    """전력 소산 맵 생성"""
    K_data = arch.calc_K()
    u = arch.u  # u 그리드
    z = u * K_data['K_total'][0, :, :]

    plt.figure(figsize=(8,6))
    plt.contourf(u, arch.vac_wavelengths, z, 
                levels=100, cmap='cividis')
    plt.colorbar(label='Normalized Dissipated Power')
    plt.xlabel('Norm. in-plane wavevector $u$')
    plt.ylabel('Wavelength (nm)')
    plt.title('2D Power Dissipation Spectrum')
    plt.show()

# 7. ETL 두께 변화 분석
def analyze_etl_thickness(simulator, thickness_range=np.arange(30, 100, 5)):
    """ETL 두께 스윕 분석"""
    ce_results = []
    for d_zn in thickness_range:
        ce = simulator.run_simulation(d_zn)
        ce_results.append(ce)
        print(f"ZnO: {d_zn} nm → CE: {ce:.1f} cd/A")
    
    # 결과 시각화
    plt.figure(figsize=(10,5))
    plt.plot(thickness_range, ce_results, 'ro-', linewidth=2)
    plt.xlabel('ZnO Thickness (nm)', fontsize=12)
    plt.ylabel('Current Efficiency (cd/A)', fontsize=12)
    plt.title('CE vs ETL Thickness', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.show()

# 8. 메인 실행 블록
if __name__ == "__main__":
    # 시뮬레이터 초기화
    simulator = TQLEDSimulator()
    
    # 2D 전력 소산 스펙트럼 (기준 두께 40nm)
    arch_std = ThinFilmArchitecture(
        layers=build_layers(40),
        dipole_positions=[0.9],
        tau=1e-6,
        PLQY=0.95,
        vac_wavelengths=simulator.vac_wavelengths,
        pl_spectrum=simulator.pl_spectrum
    )
    arch_std.load_material_data(simulator.nk_df)
    arch_std.calculate_optics()
    plot_power_dissipation(arch_std)
    
    # ETL 두께 최적화 분석
    analyze_etl_thickness(simulator)


ImportError: cannot import name 'Layer' from 'oledpy.dipole_emission' (c:\Users\minwoo\Documents\oled_project\oledpy\oledpy\dipole_emission.py)